In [1]:
#If you need a new visdom server for visualization, run this command in a terminal
# python -m visdom.server
#then navigate here http://localhost:8097

In [2]:
from models import loss
import argparse
import os
from util import util
import torch
import models
import data
import time
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
import torch.nn as nn


In [3]:
opt={#BASE OPTIONS
    #path to images (should have subfolders trainA, trainB, valA, valB, etc)
    "dataroot":"./datasets/prostate_segmentation//",

    
    #'name of the experiment. It decides where to store samples and models
    "name":"facades_pix2pix",
    
    #gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU
    "gpu_ids":[0,1],
    
    # models are saved here
    
    "checkpoints_dir":"./checkpoints",
    
    # chooses which model to use. [cycle_gan | pix2pix | test | colorization]
    "model":"pix2pixHD",
    # instance normalization or batch normalization [instance | batch | none]
    "norm":"instance",
    'use_dropout':True,
    'data_type':16,
    'verbose':True,
    'batchSize':1,
    'loadSize':1024,
    'fineSize':512,
    'label_nc':35,
    'input_nc':3,
    'output_nc':3,
    'resize_or_crop':'scale_width',
    'serial_batches':True,
    'no_flip':True,
    'nThreads':8,
    'max_dataset_size':float("inf"),
    'display_winsize':512,
    'tf_log':True,
    'netG':'global',
    'ngf':64,
    'n_downsample_global':4,
    'n_blocks_global':9,
    'n_blocks_local':3,
    'n_local_enhancers':1,
    'niter_fix_global':0,
    'no_instance':True,
    'instance_feat':True
    'label_feat':True,
    'feat_num':3,
    'load_features':True,
    'n_downsample_E':4,
    'nef':16,
    'n_clusters':10
    }
class Map(dict):
    """
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])
    """
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.items():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.items():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]
opt = Map(opt)

In [4]:
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations

for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_iters += opt.batch_size
        epoch_iter += opt.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
    model.update_learning_rate()                     # update learning rates at the end of every epoch.


dataset [AlignedDataset] was created
The number of training images = 400
initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
loading the model from ./checkpoints/facades_pix2pix/latest_net_G.pth
loading the model from ./checkpoints/facades_pix2pix/latest_net_D.pth


---------- Networks initialized -------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (3): UnetSkipConnectionBlock(
                  (model): Sequential(
              

saving the model at the end of epoch 8, iters 3216
End of epoch 8 / 200 	 Time Taken: 21 sec
learning rate = 0.0002000
(epoch: 9, iters: 84, time: 0.054, data: 0.003) G_GAN: 6.985 G_L1: 0.286 D_real: 0.035 D_fake: 0.000 
(epoch: 9, iters: 384, time: 0.071, data: 0.003) G_GAN: 0.967 G_L1: 0.393 D_real: 1.147 D_fake: 1.142 
saving the latest model (epoch 9, total_iters 3600)
saving the model at the end of epoch 9, iters 3618
End of epoch 9 / 200 	 Time Taken: 19 sec
learning rate = 0.0002000
(epoch: 10, iters: 282, time: 0.054, data: 0.004) G_GAN: 0.673 G_L1: 0.271 D_real: 0.894 D_fake: 0.653 
saving the model at the end of epoch 10, iters 4020
End of epoch 10 / 200 	 Time Taken: 17 sec
learning rate = 0.0002000
(epoch: 11, iters: 180, time: 0.067, data: 0.004) G_GAN: 0.657 G_L1: 0.309 D_real: 0.727 D_fake: 0.745 
saving the model at the end of epoch 11, iters 4422
End of epoch 11 / 200 	 Time Taken: 21 sec
learning rate = 0.0002000
(epoch: 12, iters: 78, time: 0.054, data: 0.004) G_GAN:

KeyboardInterrupt: 